 # Collision Simulate using AIS Datas
- 본 프로젝트는 AIS 데이터와 해양심판원 재결서를 바탕으로 Collision 상황을 OSM 기반 지도에 Visualization 한다
- 사용하기 앞서 추출을 위한 USER INPUT이 필요합니다. 사용자가 CUSTOM 하게 설정할 수 있습니다. 
- 또한 data의 파일들을 사용자가 Simulate 하고 싶은 data로 바꿔야 합니다.
    - data/dynamic_ais -> 사고가 발생한 날짜의 동적 AIS DATA
    - data/static_ais -> 선박의 정적 데이터
    - data/reconciliation -> 해심원의 재결서에 나온 위치, 시간 데이터

In [ ]:
# USER INPUT
    
# 데이터 추출 시간 간격 설정 ( 사고 기점으로 +- time )
time_dis = 60    
# 위도, 경도 grid 범위 잡기 ( 단위 = degree )
scale = 0.005

## Data preprocessing
- data/reconciliation 에 입력된 정보를 바탕으로 데이터를 전처리하여 dataframe에 저장하는 과정입니다. 

In [11]:
import pandas as pd
import re
from datetime import datetime, timedelta
    
def find_data_from_txt(file_path):
    accident_data = []
    dates = []
    times = []
    latitudes = []
    longitudes = []
    date_times = []
    
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    # 각 줄별로 데이터 추출
    for line in lines:
        line = line.strip()  # 줄바꿈 문자 제거
        target_year = 
        # '사고일시' 추출
        if '년' in line:
            index += 1
            date = re.findall(r'\d+', line)
            date_string = date[0] + check_digit_length(date[1]) + check_digit_length(date[2])
            time_string = date[3] + ':' + date[4] + ':' + '00'
            bar = '-'
            date_times_string = date[0] + bar + check_digit_length(date[1]) + bar + check_digit_length(date[2]) + " " + date[3] + ":" + date[4]
            times.append(time_string)
            dates.append(date_string)
            date_times.append(date_times_string)
 
        
        # '사고 장소' 추출
        if '북위' and '동경' in line:
            location = re.findall(r'\d+', line)
            latitude = convert_degrees(location[0], location[1], location[2])
            longitude = convert_degrees(location[3], location[4], location[5])
            latitudes.append(latitude)
            longitudes.append(longitude)
            
    # 데이터프레임 생성
    accident_data = pd.DataFrame({'사고날짜': dates,'사고시간': times, '위도': latitudes, '경도': longitudes, '일시': date_times})
    
    # 시간 범위 이상치 제거 
    accident_data = accident_data.drop(exception)
    

    accident_data = accident_data.reset_index(drop=True)

    return accident_data

def convert_degrees(degrees, minutes, seconds):
    degrees = float(degrees)
    minutes = float(minutes)
    seconds = float(seconds)
    
    decimal_degrees = degrees + minutes/60 + seconds/3600
    
    return decimal_degrees

def check_digit_length(number):
    if 0 <= int(number) < 10:
        return '0' + str(number)
    elif 10 <= int(number) < 100:
        return str(number)
    else:
        return None

file_path = 'reconciliation/reconciliation.txt'
accident_data = find_data_from_txt(file_path)
print(accident_data)

0
       사고날짜      사고시간         위도          경도                일시
0  20200929  15:35:00  35.501389  129.379167  2020-09-29 15:35


##  사고 날짜로 ais data 파일 이름 찾기 

In [35]:
!pip install haversine
import pandas as pd
from datetime import datetime, timedelta
import haversine as hs


startpoint_lat = 0.0
endpoint_lat = 0.0
startpoint_long = 0.0
endpoint_long = 0.0

def import_ais_data(file_paths, scale, time_dis):
    # key는 사고 일시, value는 해당 사고에 특정기준으로 추출한 dataframe
    dfs = {}
    for i, file_path in enumerate(file_paths):
        try:
            local_path = 'data/' + file_path
            df = pd.read_csv(local_path, encoding='cp949', skiprows=2, dtype={'MMSI': 'str', '일시': 'object', '위도': 'float64', '경도': 'float64', 'SOG': 'float64', 'COG': 'float64', 'Heading': 'float64'})

            print(i)
            # 데이터 전처리 수행
            accident_lat = accident_data.loc[i, '위도']
            accident_long = accident_data.loc[i, '경도']
            
            dt = datetime.strptime(accident_data.loc[i, '일시'], '%Y-%m-%d %H:%M')
            
            start_time = dt - timedelta(minutes=time_dis)
            end_time = dt + timedelta(minutes=time_dis)
            
            print(accident_lat, accident_long, start_time, end_time)
        
            startpoint_lat = accident_lat - scale
            endpoint_lat = accident_lat + scale
            startpoint_long = accident_long - scale
            endpoint_long = accident_long + scale
            loc1=(startpoint_lat, startpoint_long)
            loc2=(endpoint_lat, endpoint_long )
            print(startpoint_lat, endpoint_lat)
            print(startpoint_long, endpoint_long)

            print("distance")
            print(hs.haversine(loc1,loc2, unit = 'km'), 'km')
            # 일시 열을 시간 형식으로 변환
            df['일시'] = pd.to_datetime(df['일시'], format='%Y-%m-%d %H:%M:%S')
            
            df = df[(df['위도'] >= startpoint_lat) & (df['위도'] <= endpoint_lat) &
                    (df['경도'] >= startpoint_long) & (df['경도'] <= endpoint_long) &
                    (df['SOG'] > 0) &
                    (df['일시'].dt.time >= pd.to_datetime(start_time).time()) & (df['일시'].dt.time <= pd.to_datetime(end_time).time())]
            print(df['MMSI'])
            # check process
            print("index -> {0}. of df shape like.. ->{1}".format(i, df.shape))
            
            # AIS 수신기를 꺼놨을 경우 데이터가 없을수도 있음..  
            if (not df.empty) or (df.shape[0] > 0):
                accident_date = accident_data.loc[i, '일시']
                dfs[accident_date] = df
                
            
        except FileNotFoundError:
            print(f"File not found: {local_path}")
            continue
    return dfs
        


dfs = import_ais_data(file_paths, scale, time_dis)


0
35.50138888888889 129.37916666666666 2020-09-29 14:35:00 2020-09-29 16:35:00
35.49638888888889 35.50638888888889
129.37416666666667 129.38416666666666
distance
1.4338393906539793 km
3197113     pIbHXxwE044fWKFkiGCZeg==
3197114     pIbHXxwE044fWKFkiGCZeg==
3197115     pIbHXxwE044fWKFkiGCZeg==
3197116     pIbHXxwE044fWKFkiGCZeg==
3197117     pIbHXxwE044fWKFkiGCZeg==
                      ...           
20077226    hs74wDgQlbLcqcCfefcZSQ==
20077227    hs74wDgQlbLcqcCfefcZSQ==
20077228    hs74wDgQlbLcqcCfefcZSQ==
20077233    hs74wDgQlbLcqcCfefcZSQ==
20077239    hs74wDgQlbLcqcCfefcZSQ==
Name: MMSI, Length: 2337, dtype: object
index -> 0. of df shape like.. ->(2337, 7)


## 사고 선박 두개만을 찾아내는 함수 grid를 좁히고 사고시간과 근접하게 접근하여 회귀를 이용해 target case를 찾는다



In [36]:
def print_grouped_data(grouped_data):
    for mmsi, group in grouped_data:
        print(f"MMSI: {mmsi}")
        print(len(group))
        times = group['일시'].values
        longitudes = group['경도'].values
        latitudes = group['위도'].values
        COGs = group['COG'].values
        SOGs = group['SOG'].values
        Headings = group['Heading'].values
        
        for time, longitude, latitude, COG, SOG, Heading in zip(times, longitudes, latitudes, COGs, SOGs, Headings):
            print(f"Time: {time}, 경도: {longitude}, 위도: {latitude}, COG: {COG}, SOG: {SOG}, Headings: {Heading}")
        print()

def calc_time_range(dt, time_dis):
    dt = datetime.strptime(dt, '%Y-%m-%d %H:%M')
    
    start_time = dt - timedelta(minutes=time_dis)
    end_time = dt + timedelta(minutes=time_dis)
    return start_time, end_time

def find_by_size(grouped_data):
    print("사건 현장에 있던 선박 수:", len(grouped_data))
    # print(print_grouped_data(grouped_data))
    if len(grouped_data) < 2:
        print("데이터가 뭔가 잘못된듯? 아니면 scale 늘리기 ")
        return grouped_data
    else: 
        
        sorted_groups = grouped_data.size().sort_values(ascending=False)
        top_two_groups = sorted_groups.head(2)
        filtered_grouped_data = grouped_data.get_group(top_two_groups.index[0]), grouped_data.get_group(top_two_groups.index[1])
        
    
        combined_df = (pd.concat([filtered_grouped_data[0], filtered_grouped_data[1]], ignore_index=True)).groupby('MMSI')
        print(" filter 후의 groupe 크기-> {}".format(len(filtered_grouped_data)))
    
        return combined_df
        
def df_to_csv(df, date_time):
    datetime_obj = datetime.strptime(date_time, '%Y-%m-%d %H:%M')
    file_name = datetime_obj.strftime('%Y-%m-%d')
    file_path = 'data/' + file_name + '.csv'
    df.to_csv(file_path,mode="w",encoding='euc-kr')
    # df.to_excel(excel_writer = file_path)
    
def find_target_vessel(dfs, time_dis, search_scale):
    target = {} # key = 사건 날짜 value = Target MMSI [] 
    for key, value in dfs.items():
        print("사건 현장 내에 있는 데이터 수: ", len(value))

        
        # 데이터프레임에서 MMSI 별로 그룹화
        grouped_data = value.groupby('MMSI')

        start_time, end_time = calc_time_range(key, time_dis)
        # print("data time is -> {0}".format(value['일시']))
        # print("start time is -> {0} end time is -> {1}".format(start_time, end_time))

        filtered_data = value
        filter_by_date = accident_data[
            (accident_data['일시'] == key)
        ]
        print(filter_by_date)
        if filter_by_date.empty:
            continue
        center_long = filter_by_date['경도'].iloc[0]
        center_lat = filter_by_date['위도'].iloc[0]
        print(center_lat, center_long)
        minimize_by_timerange = value[
            (value['일시'].dt.time >= pd.to_datetime(start_time).time()) &
            # (value['SOG'] > 1) & ## 정지되어있는 부유체들 상대로 제외 
            (value['일시'].dt.time <= pd.to_datetime(end_time).time()) & 
            (value['위도'] >= (center_lat - search_scale)) & (value['위도'] <=  (center_lat + search_scale)) &
            (value['경도'] >=  (center_long - search_scale)) & (value['경도'] <= (center_long + search_scale))] 
        # 데이터프레임에서 MMSI 별로 그룹화
        grouped_data = minimize_by_timerange.groupby('MMSI')
        grouped_data = find_by_size(grouped_data)
        target_mmsis = []
        for mmsi, group in grouped_data:
            print(mmsi)
            target_mmsis.append(mmsi)
        print(target_mmsis)
        target_data = value[
            (value['MMSI'].isin(target_mmsis))
        ]
        
        df_to_csv(target_data, key)
        target[key] =  target_data # key 는 사고 일시 value 는 사고 당시 두 배에 대한 1시간 timerange의 ais 데이터들 
    return target
    
# target 찾기 위해 분으로    
time_dis = 2
search_scale = 0.001
target = find_target_vessel(dfs, time_dis, search_scale)

# # 지도 애니메이션 어떻게 그릴거임???    ㅇㅇㅇ ㅇ ㅇ ㅇ ㅇ ㅇ 

    

사건 현장 내에 있는 데이터 수:  2337
       사고날짜      사고시간         위도          경도                일시
0  20200929  15:35:00  35.501389  129.379167  2020-09-29 15:35
35.50138888888889 129.37916666666666
사건 현장에 있던 선박 수: 3
 filter 후의 groupe 크기-> 2
MjfKtEYwPcP9F9ALREXQYQ==
YgIPTK/R2Le/0wxD5aIEyQ==
['MjfKtEYwPcP9F9ALREXQYQ==', 'YgIPTK/R2Le/0wxD5aIEyQ==']


## OpenStreetMap으로 사고 상황을 시각화한다. 

In [37]:
import random
import folium


def draw_map(lines, center_lat, center_long, start_time):
    m = folium.Map(location=[center_lat, center_long], zoom_start=10)

    features = [
        {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": line["coordinates"],
            },
            "properties": {
                "times": line["dates"],
                "style": {
                    "color": line["color"],
                    "weight": line["weight"] if "weight" in line else 5,
                },
            },
        }
        for line in lines
    ]

    for feature in features:
        folium.Marker(
            location=feature["geometry"]["coordinates"][0][::-1],  # 위치 설정
            icon=folium.DivIcon(html="<div style='background-color:blue; width:15px; height:15px; border-radius:50%'></div>"),  # 배 모양 아이콘 지정
            tooltip=feature["properties"]["times"][0],  # 툴팁에 시간 정보 추가
        ).add_to(m)

    plugins.TimestampedGeoJson(
        {
            "type": "FeatureCollection",
            "features": features,
        },
        period="PT1M",
        add_last_point=True,
    ).add_to(m)

    m.save('map_with_custom_marker.html')
def random_color():
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

def convert_to_lines(df):
    lines = []
    for mmsi, group in df.groupby('MMSI'):
        coordinates = list(zip(group['경도'], group['위도']))
        dates = group['일시'].astype(str).tolist()
        color = random_color()
        line_data = {
            "coordinates": coordinates,
            "dates": dates,
            "color": color,
        }
        lines.append(line_data)
    return lines
for key, value in target.items():
        print(key)
        filtered_data = value
        filter_by_date = accident_data[
            (accident_data['일시'] == key)
        ]
        center_long = filter_by_date['경도']
        center_lat = filter_by_date['위도']
        start_time, end_time = calc_time_range(key, time_dis)
        draw_map(convert_to_lines(value), center_lat, center_long, start_time)



2020-09-29 15:35


In [38]:
import random
import folium
from folium import plugins

def create_legend(color_map):
    legend_html = """
        <div style="position: fixed;
                    bottom: 50px; left: 50px; width: 300px; height: 150px;
                    border:2px solid grey; z-index:9999; font-size:14px;
                    background-color:white; opacity:0.9;">

        <h3>Information</h3>"""

    for color, ship_type in color_map.items():
        legend_html += f'<p><span style="color:{color};">&#9632;</span> Type : {ship_type}</p>'

    legend_html += "</div>"
    return legend_html

def draw_map(lines, center_lat, center_long, start_time, color_map):
    m = folium.Map(location=[center_lat, center_long], zoom_start=15)

    features = [
        {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": line["coordinates"],
            },
            "properties": {
                "times": line["dates"],
                "style": {
                    "color": line["color"],
                    "weight": line["weight"] if "weight" in line else 5,
                },
            },
        }
        for line in lines
    ]

    for feature in features:
        folium.Marker(
            location=feature["geometry"]["coordinates"][0][::-1],
            # icon=folium.DivIcon(html="<div style='background-color:blue; width:15px; height:15px; border-radius:50%'></div>"),
            icon = folium.map.Icon(color='blue', icon_color='white', icon='flag'),
            tooltip=feature["properties"]["times"][0],
            prefix='glyphicon'
        ).add_to(m)

    plugins.TimestampedGeoJson(
        {
            "type": "FeatureCollection",
            "features": features,
        },
        period="PT1M",
        add_last_point=True,
    ).add_to(m)

    legend_html = create_legend(color_map)
    m.get_root().html.add_child(folium.Element(legend_html))

    m.save('map_with_custom_marker_and_legend.html')
def random_color():
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))
# color_map 생성


def find_ship_type(target_mmsi):
    local_path = '20200201_20210131/' + 'Static.csv'
    df = pd.read_csv(local_path,
                 encoding='cp949', 
                 dtype={'MMSI': 'str', '선박명': 'str', '선종코드': 'str', 'IMO': 'str', 'DimA': 'float64', 'DimB': 'float64', 'DimC': 'float64', 'DimD': 'float64' ,'흘수': 'float64', '추정톤수': 'float64'})

    return df[df['MMSI'] == target_mmsi]['선종코드'].iloc[0]

    
def convert_to_lines(df):
    lines = []
    for mmsi, group in df.groupby('MMSI'):
        coordinates = list(zip(group['경도'], group['위도']))
        dates = group['일시'].astype(str).tolist()
        color = random_color()
        ship_type = find_ship_type(mmsi)
        color_map[color] = ship_type
        line_data = {
            "coordinates": coordinates,
            "dates": dates,
            "color": color,
            "ship_type" : ship_type,
        }
        lines.append(line_data)
    return lines

color_map = {}

for key, value in target.items():
        print(key)
        filtered_data = value
        filter_by_date = accident_data[
            (accident_data['일시'] == key)
        ]
        center_long = filter_by_date['경도']
        center_lat = filter_by_date['위도']
        start_time, end_time = calc_time_range(key, time_dis)
        
            
        draw_map(convert_to_lines(value), center_lat, center_long, start_time, color_map)



2020-09-29 15:35


In [1]:
import gc
gc.collect()

442

##  data group by MMSI

## 항적 시간의 흐름에 따른 애니메이션 구현